# Convert dicom to png 

In [36]:
import os
import pydicom
from PIL import Image
import cv2
import numpy as np
import imageio
from skimage.transform import resize
import mritopng

def dicom_to_png(input_folder, output_folder, target_size=(128, 128)):
    # Tạo thư mục đầu ra nếu nó không tồn tại
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Lặp qua tất cả các file trong thư mục đầu vào
    for filename in os.listdir(input_folder):
        if filename.endswith(".dcm"):
            # Đọc file DICOM
            dicom_path = os.path.join(input_folder, filename)
            dicom_data = pydicom.dcmread(dicom_path)

            # Chuyển đổi DICOM sang mảng numpy
            pixel_data = np.asarray(dicom_data)
            #pixel_data = (pixel_data - np.min(pixel_data)) / (np.max(pixel_data) - np.min(pixel_data)) * 255.0
            # Resize ảnh đến kích thước mong muốn
            resized_image = resize(pixel_data, target_size, anti_aliasing=True)

            # Lưu ảnh dưới dạng PNG
            output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.png")
            imageio.imwrite(output_path, resized_image)
if __name__ == "__main__":
    # input_folder = "C:/Users/efe/OneDrive/Desktop/data/dicom"
    # output_folder = "C:/Users/efe/OneDrive/Desktop/data/png"
    
    # input_folder = "C:/Users/efe/OneDrive/Desktop/test"
    input_folder = "C:/Users/efe/OneDrive/Desktop/data/dicom/dicom_mri_T2_512"
    output_folder = "C:/Users/efe/OneDrive/Desktop/data/png//png_T2_512"
    # "C:/Users/efe/OneDrive/Desktop/dicom_data_zip/DICOM/ST00001/SE00001"
    # dicom_to_png(input_folder, output_folder)
    mritopng.convert_folder(input_folder, output_folder)


SUCCESS: C:/Users/efe/OneDrive/Desktop/data/dicom/dicom_mri_T2_512\I.001.dcm --> C:/Users/efe/OneDrive/Desktop/data/png//png_T2_512\.\I.001.dcm.png
SUCCESS: C:/Users/efe/OneDrive/Desktop/data/dicom/dicom_mri_T2_512\I.002.dcm --> C:/Users/efe/OneDrive/Desktop/data/png//png_T2_512\.\I.002.dcm.png
SUCCESS: C:/Users/efe/OneDrive/Desktop/data/dicom/dicom_mri_T2_512\I.003.dcm --> C:/Users/efe/OneDrive/Desktop/data/png//png_T2_512\.\I.003.dcm.png
SUCCESS: C:/Users/efe/OneDrive/Desktop/data/dicom/dicom_mri_T2_512\I.004.dcm --> C:/Users/efe/OneDrive/Desktop/data/png//png_T2_512\.\I.004.dcm.png
SUCCESS: C:/Users/efe/OneDrive/Desktop/data/dicom/dicom_mri_T2_512\I.005.dcm --> C:/Users/efe/OneDrive/Desktop/data/png//png_T2_512\.\I.005.dcm.png
SUCCESS: C:/Users/efe/OneDrive/Desktop/data/dicom/dicom_mri_T2_512\I.006.dcm --> C:/Users/efe/OneDrive/Desktop/data/png//png_T2_512\.\I.006.dcm.png
SUCCESS: C:/Users/efe/OneDrive/Desktop/data/dicom/dicom_mri_T2_512\I.007.dcm --> C:/Users/efe/OneDrive/Desktop/d

# Resize image and split train_test

In [1]:
from PIL import Image
import os
import random
import shutil

def reshape_and_split_images(input_folder, output_folder, train_ratio=0.8):
    # Tạo thư mục đầu ra nếu chưa tồn tại
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Danh sách các tệp ảnh trong thư mục đầu vào
    image_files = [f for f in os.listdir(input_folder) if f.endswith('.png')]

    # Số lượng ảnh để sử dụng cho tập train
    num_train = int(len(image_files) * train_ratio)

    # Xáo trộn danh sách ảnh
    random.shuffle(image_files)

    # Tạo thư mục train và test
    train_folder = os.path.join(output_folder, 'train')
    test_folder = os.path.join(output_folder, 'test')

    os.makedirs(train_folder)
    os.makedirs(test_folder)

    # Reshape ảnh và chia thành tập train và test
    for i, image_file in enumerate(image_files):
        # Đọc ảnh
        image_path = os.path.join(input_folder, image_file)
        image = Image.open(image_path)

        # Reshape ảnh về kích thước mới (128x128)
        resized_image = image.resize((128, 128))

        # Chọn thư mục dựa trên tỷ lệ train_ratio
        if i < num_train:
            output_path = os.path.join(train_folder, f"{i + 1}.png")
        else:
            output_path = os.path.join(test_folder, f"{i + 1}.png")

        # Lưu ảnh
        resized_image.save(output_path)

    print("Reshaping and splitting images completed.")

# Thực hiện hàm với thư mục đầu vào và đầu ra của bạn
input_folder = "C:/Users/efe/OneDrive/Desktop/data/png/png_T2"
output_folder = "PBL6-v1/deepgtv/db"
reshape_and_split_images(input_folder, output_folder)


Reshaping and splitting images completed.


# Add noise 

In [16]:
import numpy as np
from scipy.fftpack import fft2, ifft2, fftshift, ifftshift
from scipy.signal import fftconvolve
from bm3d import gaussian_kernel
import random
import matplotlib.pyplot as plt
import os
from PIL import Image
def get_experiment_kernel(noise_type: str, noise_var: float, sz: tuple = np.array((128, 128))):
    """
    Get kernel for generating noise from specific experiment from the paper.
    :param noise_type: Noise type string, g[0-4](w|)
    :param noise_var: noise variance
    :param sz: size of image, used only for g4 and g4w
    :return: experiment kernel with the l2-norm equal to variance
    """
    # if noiseType == gw / g0
    kernel = np.array([[1]])
    noise_types = ['gw', 'g0', 'g1', 'g2', 'g3', 'g4', 'g1w', 'g2w', 'g3w', 'g4w']
    if noise_type not in noise_types:
        raise ValueError("Noise type must be one of " + str(noise_types))

    if noise_type != "g4" and noise_type != "g4w":
        # Crop this size of kernel when generating,
        # unless pink noise, in which
        # if noiseType == we want to use the full image size
        sz = np.array([101, 101])
    else:
        sz = np.array(sz)

    # Sizes for meshgrids
    sz2 = -(1 - (sz % 2)) * 1 + np.floor(sz / 2)
    sz1 = np.floor(sz / 2)
    uu, vv = np.meshgrid([i for i in range(-int(sz1[0]), int(sz2[0]) + 1)],
                         [i for i in range(-int(sz1[1]), int(sz2[1]) + 1)])

    beta = 0.8

    if noise_type[0:2] == 'g1':
        # Horizontal line
        kernel = np.atleast_2d(16 - abs(np.linspace(1, 31, 31) - 16))

    elif noise_type[0:2] == 'g2':
        # Circular repeating pattern
        scale = 1
        dist = uu ** 2 + vv ** 2
        kernel = np.cos(np.sqrt(dist) / scale) * gaussian_kernel((sz[0], sz[1]), 10)

    elif noise_type[0:2] == 'g3':
        # Diagonal line pattern kernel
        scale = 1
        kernel = np.cos((uu + vv) / scale) * gaussian_kernel((sz[0], sz[1]), 10)

    elif noise_type[0:2] == 'g4':
        # Pink noise
        dist = uu ** 2 + vv ** 2
        n = sz[0] * sz[1]
        spec = (np.sqrt((np.sqrt(n) * 1e-2) / (np.sqrt(dist) + np.sqrt(n) * 1e-2)))
        kernel = fftshift(ifft2(ifftshift(spec)))

    else:  # gw and g0 are white
        beta = 0

    # -- Noise with additional white component --

    if len(noise_type) > 2 and noise_type[2] == 'w':
        kernel = kernel / np.sqrt(np.sum(kernel ** 2))
        kalpha = np.sqrt((1 - beta) + beta * abs(fft2(kernel, (sz[0], sz[1]))) ** 2)
        kernel = fftshift(ifft2(kalpha))

    kernel = np.real(kernel)
    # Correct variance
    kernel = kernel / np.sqrt(np.sum(kernel ** 2)) * np.sqrt(noise_var)

    return kernel


def get_experiment_noise(noise_type: str, noise_var: float, realization: int, sz: tuple) -> (np.ndarray, np.ndarray, np.ndarray):
    np.random.seed(realization)

    kernel = get_experiment_kernel(noise_type, noise_var, sz)

    half_kernel = np.ceil(np.array(kernel.shape) / 2).astype(int)

    # Crop edges
    noise = fftconvolve(np.random.normal(size=(sz[0] + 2 * half_kernel[0], sz[1] + 2 * half_kernel[1])),
                        kernel, mode='same')

    # Crop the noise to match the size of the input image
    noise = noise[half_kernel[0]:-half_kernel[0], half_kernel[1]:-half_kernel[1]]

    psd = abs(fft2(kernel, (sz[0], sz[1]), axes=(0, 1))) ** 2 * sz[0] * sz[1]

    return noise, psd, kernel
def addnoise(input_folder, noise_var: float, seed : int = 8, noise_type: str = None):
    ref = os.path.join(input_folder,"ref")
    noisy = os.path.join(input_folder,"noisy")
    if not os.path.exists(noisy):
        os.makedirs(noisy)
    image_files = [f for f in os.listdir(ref) if f.endswith('.png')]
    for i, image_file in enumerate(image_files):
        noise_type = random.choice(['g1','g2','g3','g4'])
        image_path = os.path.join(ref, image_file)
        image = np.array(Image.open(image_path)) / 255
        noise, _ ,kernel  = get_experiment_noise(noise_type, (noise_var/255)**2, seed, (128,128))
        noisy_image = image + noise
        z_rang = np.minimum(np.maximum(noisy_image, 0), 1)
        output_path = os.path.join(noisy,image_file)
        plt.imsave(output_path,z_rang,cmap='gray')
        
def addnoise_1(input_folder, noise_var_r: float,noise_var_n: float, seed : int = 8, noise_type: str = None):
    input = os.path.join(input_folder,"ref1")
    ref = os.path.join(input_folder,"ref")
    noisy = os.path.join(input_folder,"noisy")
    if not os.path.exists(noisy):
        os.makedirs(noisy)
    if not os.path.exists(ref):
        os.makedirs(ref)
    image_files = [f for f in os.listdir(input) if f.endswith('.png')]
    for i, image_file in enumerate(image_files):
        noise_type = random.choice(['g1','g2','g3','g4'])
        image_path = os.path.join(input, image_file)
        image = np.array(Image.open(image_path)) / 255
        noise, _ ,kernel  = get_experiment_noise(noise_type, (noise_var_r/255)**2, seed, (128,128))
        r = image + noise
        z_rang1 = np.minimum(np.maximum(r, 0), 1)
        output_path = os.path.join(ref,image_file)
        plt.imsave(output_path,z_rang1,cmap='gray')
        noise, _ ,kernel  = get_experiment_noise(noise_type, (noise_var_n/255)**2, seed, (128,128))
        n = image + noise
        z_rang2 = np.minimum(np.maximum(n, 0), 1)
        output_path = os.path.join(noisy,image_file)
        plt.imsave(output_path,z_rang2,cmap='gray')

# addnoise_1("deepgtv/db/train/train_40_25/",25,40,8)
addnoise("deepgtv/db/test/test_25/",25,8)
    